In [4]:
import pandas as pd
import numpy as np
from lightfm import LightFM
# import umap.umap_ as umap
import matplotlib.pyplot as plt
import scipy.sparse as ss
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import pairwise_distances_argmin

In [5]:
from datetime import datetime
from datetime import date
date = date.today()
period = "AUG"
run = f"{date}_{period}"
run

'2023-04-20_AUG'

In [5]:
%%bigquery users_tags


select event_date, event_timestamp,
    client_id, event_params
from `vertex-ai-sandbox-380023.feature_prep.watch_tags`


Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
# this I did to check what kind of design events are available

event_design = users_tags[users_tags["event_params"].str.contains("saved design")][["event_timestamp", "event_params"]]
event_design["event_params"] = event_design["event_params"].str.replace("saved design:", "")

# list(event_design["event_params"].str.split("-").explode().unique())

In [8]:
users_tags["event_params"] = users_tags["event_params"].str.lower()

tags_for_features = users_tags.copy()
list_of_tags = list(event_design["event_params"].str.split("-").explode().unique())

for watch_tag in list_of_tags:
    tags_for_features = tags_for_features[~((tags_for_features["event_params"].str.contains(watch_tag))\
                                            & (tags_for_features["event_params"].str.contains("saved design")))]
    users_tags_temp = users_tags[((users_tags["event_params"].str.contains(watch_tag))\
                                            & (users_tags["event_params"].str.contains("saved design")))]
    users_tags_temp["event_params"] = f"saved design:{watch_tag}"
    tags_for_features = pd.concat([tags_for_features, users_tags_temp])
    del users_tags_temp

del users_tags

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [9]:
tags_for_features.loc[((~tags_for_features["event_params"].str.contains("trade-in:no"))\
                    & tags_for_features["event_params"].str.contains("trade-in")), "event_params"] = "trade-in"

tags_for_features.loc\
    [(tags_for_features["event_params"].str.contains("search history") &\
     tags_for_features["event_params"].str.contains("watch")), "event_params"] = "search history:watch"

tags_for_features.loc\
    [(tags_for_features["event_params"].str.contains("search history") &\
     ~tags_for_features["event_params"].str.contains("watch")), "event_params"] = "search history:not watch"

tags_for_features.loc\
    [tags_for_features["event_params"].str.contains("manuals and downloads"), "event_params"] = "manuals and downloads"

tags_for_features.loc\
    [tags_for_features["event_params"].str.contains("vacuum cleaner"), "event_params"] = "vacuum cleaner"

tags_for_features.loc\
    [tags_for_features["event_params"].str.contains("buds2"), "event_params"] = "buds2"

tags_for_features.loc\
    [(tags_for_features["event_params"].str.contains("buds") &\
      ~tags_for_features["event_params"].str.contains("2")), "event_params"] = "buds"

tags_for_features.loc\
    [tags_for_features["event_params"].str.contains("availability"), "event_params"] = "availability"

tags_for_features.loc\
    [tags_for_features["event_params"].str.contains("home appliances"), "event_params"] = "home appliances"

tags_for_features.loc\
    [tags_for_features["event_params"].str.contains("gaming"), "event_params"] = "gaming"

tags_for_features.loc\
    [tags_for_features["event_params"].str.contains("compare"), "event_params"] = "compare"

tags_for_features.loc\
    [tags_for_features["event_params"].str.contains("dispenser"), "event_params"] = "dispenser"

tags_for_features = tags_for_features[~tags_for_features["event_params"].str.contains("result")]

remove_tags = ["tab:all", "gnb:close", "tab:products", "sticky bar:continue"]
tags_for_features = tags_for_features[~tags_for_features["event_params"].isin(remove_tags)]

tags_for_features.loc[tags_for_features["event_params"].str.contains("stock alert"), "event_params"] = "stock alert"

In [10]:
clrs = tags_for_features[tags_for_features["event_params"].str.contains("colour")]
clrs.groupby(["event_params"]).event_timestamp.count().reset_index().sort_values(by="event_timestamp", ascending=False)

,event_params,event_timestamp
62,colour:graphite,61884
125,colour:silver,45439
64,colour:gray titanium,30230
14,colour:black,25472
17,colour:black titanium,16003
...,...,...
132,colour:turquoise green,1
58,colour:flamingo pink,1
1,accordion:what do the led notification light c...,1
80,colour:metallic blue,1


In [11]:
tags_gr = tags_for_features.groupby(["event_params"])\
    .event_timestamp.count()\
    .reset_index()\
    .sort_values(by="event_timestamp", ascending=False)


In [12]:
short_tags_list = list(tags_gr[tags_gr["event_timestamp"] > 1000]["event_params"])
# len(short_tags_list)
tags_for_features_short = tags_for_features[tags_for_features["event_params"].isin(short_tags_list)]
tags_for_features_short.event_params.nunique()

642

In [13]:
# preparing features for tags
tags_to_pivot = tags_for_features_short[["client_id", "event_timestamp", "event_params"]].drop_duplicates()\
    .reset_index(drop=True)

del tags_for_features_short

In [14]:
# preparing features for tags

pivot_tags = pd.pivot_table(tags_to_pivot, index=["client_id"],
                            columns=["event_params"], values="event_timestamp",
                            aggfunc="count", fill_value=0).reset_index()

del tags_to_pivot

In [15]:
pivot_tags.to_parquet("AUG_tags_per_user.pq")

In [13]:
pivot_tags = pd.read_parquet("AUG_tags_per_user.pq")
pivot_tags.shape

(326197, 643)

In [4]:
sparse_tags = ss.csr_matrix(pivot_tags.drop(columns=["client_id"]).values)
sparse_tags

<326197x642 sparse matrix of type '<class 'numpy.int64'>'
	with 3155261 stored elements in Compressed Sparse Row format>

In [5]:
if "model_tags" in locals():
    del model_tags
n_comp = 25
model_tags = LightFM(loss='warp', random_state=42, no_components=n_comp)
model_tags.fit(sparse_tags, epochs=100, num_threads=1)
user_emb_tags = model_tags.user_embeddings

In [6]:
emb_tags = pd.DataFrame(user_emb_tags)
for col in emb_tags.columns:
    emb_tags = emb_tags.rename(columns={col: f"col_{col}"})

In [7]:
emb_tags.to_parquet(f"emb_tags_{n_comp}.pq")

In [11]:
n_comp = 25
emb_tags_25 = pd.read_parquet(f"emb_tags_{n_comp}.pq").values

In [12]:
emb_tags_25

array([[-0.0147962 , -0.39761937, -0.26960868, ...,  0.1258422 ,
        -0.16644514, -0.31131908],
       [ 0.33503792, -0.19317514, -0.21924737, ...,  0.04283247,
         0.21233997,  0.04810783],
       [ 0.11598038,  0.19888817, -0.508257  , ...,  0.64314544,
         0.6212508 , -0.00304114],
       ...,
       [-0.43279177,  0.21961479,  0.01257806, ..., -0.08636177,
         0.34774768,  0.03724869],
       [ 0.08224156, -0.03003123,  0.1159418 , ..., -0.24202669,
         0.36805233,  0.1645947 ],
       [ 0.18292034, -0.3916358 ,  0.6435098 , ...,  0.36850867,
         0.2953449 ,  0.3049081 ]], dtype=float32)

In [14]:
import umap.umap_ as umap

if "umap_model" in locals():
    del umap_model

# defining the UMAP model
umap_model = umap.UMAP(n_components=2, random_state=21, metric='manhattan', n_neighbors=30)

In [ ]:

embedding = umap_model.fit_transform(emb_tags_25)

# plotting
# plt.scatter(embedding[:, 0], embedding[:, 1])
# plt.show()

In [11]:
user_emb_page = pd.read_parquet("emb_page_ev_15.pq").values
user_emb_tags = pd.read_parquet("emb_tags_25.pq").values
user_emb_dev = pd.read_parquet("emb_devices_7.pq").values

In [14]:
embedded_tags = pd.concat([pivot_tags[["client_id"]], pd.DataFrame(user_emb_tags)], axis=1)
tag_columns = list()
tag_columns.append("client_id")
for i in range(embedded_tags.shape[1]-1):
    new_entry = f"embed_tag_{i}"
    tag_columns.append(new_entry)
embedded_tags.columns = tag_columns

In [22]:
embedded_tags.to_parquet("embed_tags_25.pq")

In [15]:
events_pages = pd.read_parquet("3m_events_pages_features.pq")
# user_emb_page = pd.read_parquet("emb_page.pq").values

In [16]:
embedded_page = pd.concat([events_pages[["client_id"]], pd.DataFrame(user_emb_page)], axis=1)
page_columns = list()
page_columns.append("client_id")
for i in range(embedded_page.shape[1]-1):
    new_entry = f"embed_page_{i}"
    page_columns.append(new_entry)
embedded_page.columns = page_columns

In [17]:
# user_emb_dev = pd.read_parquet("dev_emb.pq").values
user_device_features = pd.read_parquet(f"users_device_features_{run}.pq")

In [18]:
embedded_dev = pd.concat([user_device_features[["client_id"]], pd.DataFrame(user_emb_dev)], axis=1)
dev_columns = list()
dev_columns.append("client_id")
for i in range(embedded_dev.shape[1]-1):
    new_entry = f"embed_dev_{i}"
    dev_columns.append(new_entry)
embedded_dev.columns = dev_columns

In [19]:
user_embeddings_full = embedded_dev.merge(embedded_page, on="client_id", how="outer")\
    .merge(embedded_tags, on="client_id", how="outer")\
    .fillna(0)


In [20]:
users_num = pd.read_parquet(f"users_numerical_activity_{run}.pq")

In [21]:
user_embed_cat_num = users_num.merge(user_embeddings_full, on="client_id", how="inner")

In [22]:
user_embed_cat_num.to_parquet(f"{run}_hyp_embeddings_with_num.pq")

In [7]:
user_embed_cat_num = pd.read_parquet(f"{run}_embeddings_with_num.pq")

In [23]:
F = user_embed_cat_num.drop(columns="client_id")
F = StandardScaler().fit_transform(F)

In [ ]:
user_emb_dev_7
emb_page_ev_15
user_emb_tags_25

In [24]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
if "mbk_new" in locals():
    del mbk_new
    
for n_clusters in range(2, 15):
    print(f"running {n_clusters} number of clusters")
    mbk_new = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, n_init=10, batch_size=512)
    cluster_labels = mbk_new.fit_predict(F)
    # silhouette_avg = silhouette_score(F, cluster_labels)
    # print("For n_clusters = {}, the average silhouette score is {:.2f}".format(n_clusters, silhouette_avg))
    ch_score = calinski_harabasz_score(F, cluster_labels)
    db_score = davies_bouldin_score(F, cluster_labels)
    print("For n_clusters = {}, the average calinski_harabasz score is {:.2f}".format(n_clusters, ch_score))
    print("For n_clusters = {}, the average davies_bouldin score is {:.2f}".format(n_clusters, db_score))

running 2 number of clusters
For n_clusters = 2, the average calinski_harabasz score is 951.87
For n_clusters = 2, the average davies_bouldin score is 3.54
running 3 number of clusters
For n_clusters = 3, the average calinski_harabasz score is 18933.57
For n_clusters = 3, the average davies_bouldin score is 4.03
running 4 number of clusters
For n_clusters = 4, the average calinski_harabasz score is 17945.50
For n_clusters = 4, the average davies_bouldin score is 4.15
running 5 number of clusters
For n_clusters = 5, the average calinski_harabasz score is 16272.99
For n_clusters = 5, the average davies_bouldin score is 3.63
running 6 number of clusters
For n_clusters = 6, the average calinski_harabasz score is 14865.96
For n_clusters = 6, the average davies_bouldin score is 3.64
running 7 number of clusters
For n_clusters = 7, the average calinski_harabasz score is 14586.73
For n_clusters = 7, the average davies_bouldin score is 3.52
running 8 number of clusters
For n_clusters = 8, the a

In [26]:
if "mbk" in locals():
    del mbk

# sample_df = user_embeddings_full.sample(frac=0.5, random_state=1)
F = user_embed_cat_num.drop(columns="client_id")
F = StandardScaler().fit_transform(F)

n_clusters = 4
batch_size = 256

mbk = MiniBatchKMeans(n_clusters=n_clusters, random_state=42, n_init=10, batch_size=batch_size)
mbk.fit(F) 
mbk_means_cluster_centers = np.sort(mbk.cluster_centers_, axis = 0) 
mbk_means_labels = pairwise_distances_argmin(F, mbk_means_cluster_centers) 
yhat_4 = mbk.predict(F)

# plt.figure(figsize=(12,8))
# plt.scatter(embedding[:, 0], embedding[:, 1], c=yhat_7)
# plt.title(f"Amount of clusters is {n_clusters}, batch size is {batch_size}")
# plt.colorbar()
# plt.show()

In [27]:
data = user_embed_cat_num.copy()

In [28]:
data["cluster_4"] = yhat_4
data.to_parquet("clustered.pq")

In [29]:
if "umap_minikmeans" in locals():
    del umap_minikmeans

umap_minikmeans = umap.UMAP(n_components=2, random_state=1, metric="manhattan", n_neighbors=10)
embedding = umap_minikmeans.fit_transform(F)

plt.figure(figsize=(15,10))
plt.scatter(embedding[:, 0], embedding[:, 1], c=yhat_4)
plt.colorbar()
plt.show()

NameError: name 'umap' is not defined